In [ ]:
import json
with open('results.json','r') as f:
    data = json.load(f)
data = data[0]
res = {}
for i in range(len(data)):
    res[i+1] = {'lat':data[i]['latitude'], 'lon':data[i]['longitude']}
    
with open('parking_lots.json','w') as f:
    json.dump(res, f, indent=2)

In [2]:
import json 
from minimum_cost import MinimumCost as mc
with open('parking_lots.json','r') as f:
    parkings = json.load(f)

for n, data in parkings.items():
    lat = data['lat']
    lon = data['lon']
    cords = (lat,lon)
    address = mc.c2a(cords)
    data['address'] = address

with open('pl2.json','w') as f:
    json.dump(parkings, f, indent=2) 

In [ ]:
from distances import Distances as dst
import json
import osmnx as ox 
import networkx as nx
import requests
from time import perf_counter
import numpy as np
import numpy as np

def find_roads_within_range(lat, lon):
    # Define a bounding box around the coordinates with a specified distance in meters
    north, south, east, west = ox.utils_geo.bbox_from_point((lat, lon), dist=4000)
    
    # Fetch the graph with a custom filter if provided
    filter1 = '["highway"~"motorway|trunk|primary|secondary|motorway_link|trunk_link|primary_link"]'
    filter2 = '["highway"~"motorway|trunk|primary|motorway_link|trunk_link|primary_link"]'
    graph = ox.graph_from_bbox(north, south, east, west, custom_filter=filter1)
    
    # Convert the graph to GeoDataFrame of edges
    edges_gdf = ox.graph_to_gdfs(graph, nodes=False, edges=True)
    
    # Filter edges to include only those with non-empty name attribute
    named_edges = edges_gdf.dropna(subset=['name'])
    
    # Flatten the lists in the 'name' column
    named_edges['name'] = named_edges['name'].apply(lambda x: x[0] if isinstance(x, list) else x)
    
    # Return the GeoDataFrame containing roads within the range and matching the custom filter
    return list(set(named_edges['name']))

with open('updated_parkings.json','r') as f:
    parkings = json.load(f)
    
for n, data in parkings.items():
    lat = data['lat']
    lon = data['lon']
    lst = find_roads_within_range(lat, lon)
    num = len(lst)
    parkings[n]['roads_num'] = num

with open('u2p.json', 'w') as f:
    json.dump(parkings, f, indent=2)

In [2]:
import json

def update_json_with_addresses(location_file, address_file, output_file):
    # Load location data
    with open(location_file, 'r') as file:
        locations = json.load(file)
    
    # Load address data
    with open(address_file, 'r') as file:
        addresses = json.load(file)
    
    # Update locations with address data
    for key, value in locations.items():
        if key in addresses:
            # Assuming that the addresses JSON only contains an 'address' key per id
            locations[key]['address'] = addresses[key]['address']
    
    # Write the updated JSON data to a file
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(locations, file, ensure_ascii=False, indent=4)

# Example usage of the function
location_file = 'u2p.json'  # Your original location data JSON
address_file = 'pl2.json'  # The JSON file with address information
output_file = 'updated_locations.json'  # File where the updated JSON will be stored

update_json_with_addresses(location_file, address_file, output_file)
